In [1]:
from datasets import Contains1
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Contains1(num_points=1000)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[ 5,  8, 12, 11, 16,  1, 13, 11, 13, 17],
         [15,  4,  4,  8,  6, 15, 14, 15, 15, 19],
         [14,  5, 13,  9, 17, 15, 17, 15,  9, 16],
         [16,  1,  5,  3,  8, 15,  1,  5, 18,  8],
         [ 4,  7, 11, 11, 18, 12,  5,  9,  3,  9],
         [ 2, 12, 16, 14, 17,  2,  1,  1, 11, 11],
         [10,  9, 14, 12, 15, 14,  2,  8,  4, 10],
         [19,  1, 12,  6, 15, 10,  3, 17,  2, 14],
         [ 7,  8,  3, 12,  4, 19,  8, 12, 11, 15],
         [ 3, 12, 11, 12, 14,  2,  4,  7, 18, 16]]),
 tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 0]))

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 0    360
1    340
dtype: int64
Val data:
 1    62
0    38
dtype: int64
Test data:
 0    102
1     98
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-03-24 01:42:57.141852: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [6]:
np.log(2)

0.6931471805599453

In [7]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6932724118232727, grad: 0.0),
 node(name: +, val: [[0.00060837 0.00336968]
  [0.0007587  0.00355593]
  [0.00069366 0.00383811]
  [0.00060835 0.00337013]
  [0.00060839 0.00337033]
  [0.00045666 0.00412864]
  [0.00069371 0.00383842]
  [0.00065592 0.00371523]
  [0.00079444 0.00307746]
  [0.00063114 0.00404991]
  [0.00065594 0.00371517]
  [0.00069379 0.00383843]
  [0.00069366 0.00383838]
  [0.0005831  0.00396602]
  [0.00049469 0.00409725]
  [0.00050911 0.00414495]
  [0.00075872 0.00355627]
  [0.00045662 0.0041285 ]
  [0.00069563 0.00368552]
  [0.00069556 0.00368549]
  [0.00060827 0.00336927]
  [0.00090493 0.00221035]
  [0.00072354 0.0038369 ]
  [0.00069565 0.00368602]
  [0.00069375 0.00383822]
  [0.00064152 0.00401879]
  [0.00065591 0.00371515]
  [0.00042537 0.00410883]
  [0.00042526 0.00410871]
  [0.00069373 0.00383871]
  [0.00063115 0.0040499 ]
  [0.00065532 0.00381957]
  [0.00090494 0.00221023]
  [0.00042538 0.00410876]
  [0.0004566  0.00412879]
  [0.00090497 0.002